In [12]:
# UT-TOR-DATA-PT-01-2020-U-C Week 11
# Web Design Challenge
# Creates a stack chart showing results of all the parties on all the elections
# (c) Boris Smirnov

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import utilities as u

In [14]:
years_lst = [2004, 2006, 2008, 2011, 2015, 2019]
party_lst = ['LIB', 'CPC', 'NDP', 'BQ', 'GRN']

# I have custom names only for the parties in the list. Very convenient
p_df = u.parties_df[['Id', 'Custom Name']].dropna()
party_names = dict(zip(p_df['Id'], p_df['Custom Name']))

party_colors = {
    'LIB': '#EA6D6A', # red
    'CPC': "#6495ED", # blue
    'NDP': "#F4A460", # orange
    'BQ' : "#87CEFA", # light blue
    'GRN': "#99C955" # green
}

In [15]:
# DataFrame with a list of candidates who almost made it
almost_made_it_df = pd.DataFrame()

for year in reversed(years_lst):
    src_file = f"data{year}.csv"
    input_df = pd.read_csv(src_file)

    # Independent candidates make life more difficult, because they sometimes win
    # and there can be more then one in a riding
    # Thus, I explicitly create unique id for each candidate
    # Fortunately none of them got to the second or first place
    input_df['Id'] = input_df.index

    # Now, when each candidate is uniquely identified, we can use groupby (which ruins all indexes)
    winners_df = input_df.groupby('FED Id').first()

    # Remove winners from the source dataset
    input_df.drop(index=winners_df['Id'], inplace=True)

    # Find candidates who came second
    seconders_df = input_df.groupby('FED Id').first()
    seconders_df

    # Combine winners and seconders
    almost_df = pd.DataFrame.merge(winners_df, seconders_df, how='left', left_index=True, right_index=True)
    almost_df.drop(columns=[
        'Candidate Pid_x', 'Id_x', 'Province Id_y', 'Province Name_y', 'FED Name_y', 'Winning Pid_y', 'Ballots_y', 'Id_y'
    ], inplace=True)
    almost_df.columns = ['Province Id', 'Province Name', 'FED Name', 'Pid 1', 'Ballots', 'Votes 1', 'Pid 2', 'Votes 2']

    # Calculate differences in votes number and select those where it is <= 1% of ballots
    almost_df['Diff'] = almost_df['Votes 1'] - almost_df['Votes 2']
    almost_df['%'] = almost_df['Diff'] * 100 / almost_df['Ballots']
    almost_df.drop(index=almost_df[almost_df['%'] > 1].index, inplace=True)

    # Add year and reset index
    almost_df = pd.DataFrame([year] * len(almost_df), columns=['Year']).\
        merge(almost_df.reset_index(), left_index=True, right_index=True)

    # Append to result
    if not len(almost_made_it_df):
        almost_made_it_df = almost_df
    else:
        almost_made_it_df = pd.concat([almost_made_it_df, almost_df], ignore_index=True)

In [16]:
almost_made_it_df

,Year,FED Id,Province Id,Province Name,FED Name,Pid 1,Ballots,Votes 1,Pid 2,Votes 2,Diff,%
0,2019,12003,NS,Nova Scotia,Cumberland--Colchester,LIB,45897,16672,CPC,16219,453,0.986993
1,2019,24028,QC,Quebec,Hochelaga,LIB,53944,18008,BQ,17680,328,0.608038
2,2019,24059,QC,Quebec,Québec,LIB,55249,18047,BQ,17722,325,0.588246
3,2019,35046,ON,Ontario,Kitchener--Conestoga,LIB,51896,20480,CPC,20115,365,0.703330
4,2019,35087,ON,Ontario,Richmond Hill,LIB,50733,21804,CPC,21592,212,0.417874
...,...,...,...,...,...,...,...,...,...,...,...,...
61,2004,48011,AB,Alberta,Edmonton--Beaumont,LIB,41178,17555,CPC,17421,134,0.325416
62,2004,59003,BC,British Columbia,Burnaby--New Westminster,NDP,41915,14061,LIB,13732,329,0.784922
63,2004,59017,BC,British Columbia,New Westminster--Coquitlam,CPC,47870,15693,NDP,15580,113,0.236056
64,2004,59031,BC,British Columbia,Vancouver Island North,CPC,53038,18733,NDP,18250,483,0.910668


In [17]:
# Generating tables for political parties, to show how many more seats they could get

for party in party_lst:

    losers_df = almost_made_it_df[almost_made_it_df['Pid 2'] == party].copy()
    if not len(losers_df): # Green Party never lost by small margin
        continue
    losers_df['Party'] = losers_df['Pid 1'].map(lambda p: party_names[p])
    losers_df['%'] = losers_df['%'].map(lambda p: "{:.2f}%".format(p))

    result_df = losers_df[['Year', 'Province Name', 'FED Name', 'Party', 'Diff', '%']]
    result_df.columns = ['Year', 'Province name', 'Riding', 'Lost to', 'By # votes', '% of ballots']
    result_df.set_index(['Year', 'Province name'], inplace=True)

    # Save the results
    result_df.to_html(
        f"../html/{party}_almost_table.html",
        border=0, justify="left",
        classes="table table-striped table-sm small")


In [45]:
# Generating tables by elections, to show how seats could be redistributed

for year in years_lst:

    losers_df = almost_made_it_df[almost_made_it_df['Year'] == year].copy()
    losers_df['Losing party'] = losers_df['Pid 2'].map(lambda p: party_names[p])
    losers_df['Lost to'] = losers_df['Pid 1'].map(lambda p: party_names[p])
    losers_df['%'] = losers_df['%'].map(lambda p: "{:.2f}%".format(p))
    losers_df.sort_values(by=['Losing party', 'Province Name', 'FED Name'], inplace=True)

    result_df = losers_df[['Losing party', 'Province Name', 'FED Name', 'Lost to', 'Diff', '%']]
    result_df.columns = ['Losing party', 'Province name', 'Riding', 'Lost to', 'By # votes', '% of ballots']
    result_df.set_index(['Losing party', 'Province name'], inplace=True)

    # Save the results
    result_df.to_html(
        f"../html/election{year}_almost_table.html",
        border=0, justify="left",
        classes="table table-striped table-sm small")